# Indicateurs et ratios

## Calcul des indicateurs et ratios

In [ ]:
import pandas as pd
import yaml
import requests

In [ ]:
def get_eur_to_usdc_rate():
    # API publique pour le taux EUR/USDC via CoinGecko
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {"ids": "usd-coin", "vs_currencies": "eur"}  # ID pour USDC  # Contre EUR

    response = requests.get(url, params=params)
    data = response.json()

    # Récupère le taux EUR -> USDC
    rate = 1 / data["usd-coin"]["eur"]  # Inverser car API donne EUR -> USDC
    return rate


RATIO_EUR_USDC = get_eur_to_usdc_rate()


def convert_eur_to_usdc(euros):
    return euros * RATIO_EUR_USDC

In [ ]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

DATA_FOLDER = config["DATA_FOLDER"]
TMP_FOLDER = config["TMP_FOLDER"]
RATIO_TRADE_MAX = config["RATIO_TRADE_MAX"]

In [ ]:
revenus_depenses = pd.read_csv(DATA_FOLDER + "revenus-depenses.csv")
revenus_depenses["Montant"] = pd.to_numeric(revenus_depenses["Montant"])
revenus_depenses = revenus_depenses.sort_values(by=["Montant"], ascending=False)

In [ ]:
patrimoine = pd.read_csv(TMP_FOLDER + "patrimoine.csv")

In [ ]:
depense = revenus_depenses.loc[revenus_depenses["Type"] == "depense"]
depense_total = depense["Montant"].sum()

In [ ]:
revenu_total = revenus_depenses.loc[
    revenus_depenses["Type"] == "revenu", "Montant"
].sum()
epargne = revenu_total - depense_total
taux_depargne = epargne / revenu_total

pourcentage_don = depense["Montant"].values[1] / revenu_total

epargne_precaution = depense_total *3 
taux_depense_variable = (
    revenus_depenses.loc[
        (revenus_depenses["Nom"] == "Courante")
        & (revenus_depenses["Type"] == "depense"),
        "Montant",
    ].sum()
    / revenu_total
)
taux_depense_mensuelle = (
    revenus_depenses.loc[
        (revenus_depenses["Nom"] != "Courante")
        & (revenus_depenses["Type"] == "depense"),
        "Montant",
    ].sum()
    / revenu_total
)

In [ ]:
print("Taux d'épargne: {:.2%}".format(taux_depargne))
print("Taux de dépense variable: {:.2%}".format(taux_depense_variable))
print("Taux de dépense mensuelle: {:.2%}".format(taux_depense_mensuelle))
print("Pourcentage de don: {:.2%}".format(pourcentage_don))
print("Epargne de précaution: {:.2f}".format(epargne_precaution))

# Crypto

In [ ]:
trade_max_crypto = (
    patrimoine.loc[patrimoine["Type"] == "numerique", "Plus-Moins Value"].sum()
    * RATIO_TRADE_MAX
)
trade_crypto_max_usdc = convert_eur_to_usdc(trade_max_crypto)

In [ ]:
print("Mon trade max crypto est de {:.2f}€".format(trade_max_crypto))
print("Soit {:.2f}USDC".format(trade_crypto_max_usdc))